In [2]:
import math as m
import numpy as np
import random as rnd
import re
import itertools as it
import pandas as pd

# 1.1

Consider a standard, well-shuffled deck of cards. What is the probability that the 4 Aces are all adjacent? Here, define ‘adjacent’ as taking four consecutive positions; i.e., the 1st, 2nd, 3rd and 4th cards in the deck. The definition is not circular: the first and last cards in the deck are not considered adjacent.

In [69]:
# Let's try to simulate first
# Cards will just be represented by 1 to 52, let's say 1:4 represents the 4 Aces
cards = np.arange(1,53)

# Simulating shuffles
# Take rolling sums of cards  1+2+3+4 = 10, look for 10
# The only way to sum to 10 is with 1:4
# sum == (convolve with [1,1,1,1])
num_sims = 100000

conseq = np.zeros(num_sims)

for i in range(num_sims):
    np.random.shuffle(cards)
    conseq[i] = 10 in np.convolve(cards, np.ones(4), mode = 'valid')

conseq.mean()

0.00017000000000000001

### 1.1 Analytical Solution

There are 52! ways to order the entire deck of cards, this is the event space.  
  
Split the deck into two 'groups' of cards: aces (4) and non-aces (48).  
  
There are 4! ways to order the aces and 48! ways to order the non-aces.  
  
4! * 48! ways to mix and match these 'groups', independent orderings of aces/non-aces.  
  
We can drop the group of aces before (1), after (1), or in-between (47) the other cards.  
So there are 49 places we can insert the aces.

### $\frac{\text{(Places to insert aces) * (Ways to order aces * Ways to order non-aces)}}{\text{Ways to order all cards}} = \frac{49\cdot(4!\cdot48!)}{52!} = \frac{1}{5525}$

The numerator basically answers:  
*How many different ways are there to construct four consecutive aces in a 52 card deck?*  

The denominator answers:  
*How many ways are there to order 52 cards?*  

The ratio gives a probability of observing the aforementioned event in the event space.

# 1.2

a. There are eight schools in the Ivy League (Harvard’s athletic conference): Harvard, Yale, Princeton, Brown, Columbia, Dartmouth, Cornell and Penn. The academic rankings of these schools is often a topic of interest and controversy. How many different ways are there to rank these schools?

b .Sometimes, Harvard, Yale and Princeton are referred to as the ‘Big 3’, and are often grouped together as schools because of their extensive similarities. If indeed these three schools were identical - that is, the ordering ‘Harvard Yale Princeton’ is taken as identical to the ordering ‘Yale Princeton Harvard’ - how many ways would there now be to rank the Ivies?

### 1.2.a Analytical Solution:  

This question boils down to just the permutations, there are 8! = 40,320 possible rankings.

### 1.2.b Analytical Solution:

8! overcounts the total number of rankings, we need to remove all of the ways to rank 3 schools. We're overcounting by 3! since those permutations all count as one group now: 
## $\frac{8!}{3!} = 6,720$

# 1.3

a. You are tasked with dividing 20 kids into two kickball teams at recess. Games have been very even in the past, so you decide to make things more interesting by giving 1 team 9 players and the other team 11 players. How many ways could you make these teams?

b. Can you write your answer to part (a) in a different format?

c. After the first game, the team with 9 people complained because of the inherent disadvantage. For the second game, you decide to again put 10 people on each team. How many ways can you do this?

### 1.3.a + 1.3.b Analytical Solution:  

We first need to make a group of size 9 from 20 without replacement where order doesn't matter. The other 11 kids naturally fall into the second group. You get the same answer if you construct the group of 11 first and make the others go into the group of 9.

This has to do with the symmetry of the binomial coefficient:  
## $\binom{N}{K} = \binom{N}{N-K}$

## $\binom{20}{11} = \binom{20}{9} = 167,960$

### 1.3.c Analytical Solution:

The same logic applies as above except now we're putting together a team of size 10. We actually end up overcounting by a factor of two. Imagine picking kids [1..10] for team one first, this implies [11..20] fall into the second group. Going in reverse, picking [11..20] first, leaves you with the same exact team membership but the binomial coefficient counts these as distinct events.   

## $\binom{20}{10}/2 = 92,378$

# 1.4
For this problem, assume a normal, well-shuffled 52 card deck. You are dealt 5 cards (called a ‘hand’) randomly from the deck.

a. The best hand in Poker is a royal flush: the 10, Jack, Queen, King and Ace of the same suit. What is the probability that you are dealt a royal flush?

b. What is the probability that you are dealt a 3 of a kind (getting exactly 3 of the same value, like three jacks)?  

In [23]:
# Takes a very long time, rare event
num_sims = 1000000

cards = np.arange(1,53)

#

flush = np.zeros(num_sims)

for i in range(num_sims):

    # Randomly draw 5 cards
    np.random.shuffle(cards)
    
    draw = cards[0:5]
    
    # Check if they are of the same suit
    if np.unique((draw - 1)//13).size == 1:
        # Check if they are a particular group of 5 cards in the deck
        # We arbitrarily asign 1:5 to the 10, Jack, Queen, King, and Ace
        if np.all(np.sort((draw - 1) % 13) == np.array([1,2,3,4,5])):
            flush[i] = 1
            
np.mean(flush)

1.9999999999999999e-06

### 1.4.a Analytical Solution:

There are 20 possible ways to start a 'winning' sequence. Five cards * four suits.   
For each consecutive 'pick' there are 5 minus 'pick' cards left to complete the royal flush. [4..3..2..1..]   
As you draw each 'pick' out of the deck there is one less card in the whole deck.  [51..50..49..48..]

## $\frac{20}{52} * \frac{4}{51} * \frac{3}{50} * \frac{2}{49} * \frac{1}{48} = \frac{1}{649,740}$

Another way to think about is that there are $\binom{52}{5}$ ways to pick five card combinations out of the deck. There are four suits, which means that there are only four ways to get a royal flush out of all possible five card orderings.  

## $\frac{4}{\binom{52}{5}} = \frac{1}{649,740}$

### 1.4.b Analytical Solution:  

There are $\binom{52}{5}$ possible hands to begin with.  

First, we select what card we want three of [2..Ace] - there are 13 mutually exclusive possibilities. Let's say we want three aces.  

Next, there are four suits but we only need three of the four possible cards. There are $\binom{4}{3}$ ways to construct this group of three. This accounts for three out of five cards in our hand. Say we have [Ace/Spades, Ace/Clubs, Ace/Hearts].  

Once we have picked three out of the four cards we want, there are 52 - 3 - 1  = 48 possible cards left to pick from. If we have 3 aces as our three-of-a-kind that means there's an extra ace still in the deck, Ace/Diamonds in this case. We have to adjust this one out since picking it will give us four-of-a-kind. We now have four cards.   

The fifth card can't be the same card as the one used for our three-of-a-kind. It also can't be the same as the fourth card, which would be a full house (three-of-a-kind + two-of-a-kind). This leaves us with 48 - 1 - 3 = 44 choices for a fifth card. 

Another point to make here is that the order in which we observe the fourth and fifth cards doesn't matter. The accounting for the five cards above overcounts by a factor of two since it treats both of these cases as distinct: [A,A,A,3,4] and [A,A,A,4,3]. 


### $\frac{\text{(Card we want) * (Ways to get three of the card we want) * [(Ways to get fourth card) * (Ways to get fifth card)]/(Adjustment)}}{\text{Possible five card hands}}$

## $\frac{13 * \binom{4}{3} * ([48 * 44]/2)}{\binom{52}{5}} \approx 2.1128\%$

# 1.5

Tony has 5 meetings to schedule this business week (Monday through Friday). Define a ‘permutation’ here as a schedule of meetings by day (i.e., two meetings on Monday, one meeting on Thursday and two meetings on Friday is one permutation). If meetings are indistinguishable, how many permutations are there if Tony does not want to have all five meetings on a single day?

### 1.5 Analytical Solution:

There are two ways to approach this problem. The first is by [paritioning](https://math.stackexchange.com/questions/217597/number-of-ways-to-write-n-as-a-sum-of-k-nonnegative-integers) the number 5, while allowing for zeros to be included in the partitioning scheme. We eliminate the seventh partition [0,0,0,0,5] since we don't want all the meetings on the same day. 

We then look at all the permutations of each partition's elements (5!) and adjust out the fact that some numbers are repeated. For example [0,0,1,1,3] repeats both 0 and 1 twice. Adjusting 5! by dividing out (2! * 2!) removes the overcounting of these identical duplicates. Summing up the unique permutations of each partition (with zeros allowed) gives the answer.

In [4]:
comb1 = len(set(it.permutations([1,1,1,1,1]))) # 5!/5!
comb2 = len(set(it.permutations([0,1,1,1,2]))) # 5!/3!
comb3 = len(set(it.permutations([0,0,1,2,2]))) # 5!/(2!*2!)
comb4 = len(set(it.permutations([0,0,1,1,3]))) # 5!/(2!*2!)
comb5 = len(set(it.permutations([0,0,0,1,4]))) # 5!/3!
comb6 = len(set(it.permutations([0,0,0,2,3]))) # 5!/3!

print(comb1)
print(comb2)
print(comb3)
print(comb4)
print(comb5)
print(comb6)

comb1 + comb2 + comb3 + comb4 + comb5 + comb6

1
20
30
30
20
20


121

### 1.5 Analytical Solution Continued

The second approach involves thinking about sampling the days with replacement. A valid sampling could be [Monday, Monday, Tuesday, Wednesday, Friday]. The number of ways we could sample n things into groups of size k with replacement can be represented with the Bose-Einstein result:

# $\binom{Days + Meetings - 1}{Meetings} = \binom{5 + 5 - 1}{5} = 126$

We then need to adjust out the fact we don't want 5 meetings on the same day. To do this we subtract out the 5 possible ways this can happen to get a final answer of 121.

# 1.6
Consider a state that has 5 different counties (a ‘county’ is a collection of towns: for example, the town Burlington, Connecticut is in Hartford County), and each county has 6 different towns. Imagine that you want to visit every town in the state; however, you don’t want to visit each county more than once (you can fly from any town to any other town in the state). Define a permutation as one specific ordering of the 30 visits you make (one to each town). How many permutations are there?

### 1.6 Analytical Solution:

Just to clarify, the problem means that once you start in a county you must visit every town. This essentially places a restriction on the 30! total trips that are possible. We can think of this problem as constructing a table with columns [A, B, C, D, E] to represent the counties and rows enumerated 1:6 to represent the towns. A trip is a randomized version of this table where we start at [1,1] and move down the rows until [1,6]. Then we jump over to the column at the right. This continues until the last entry - [6,5].  

Here's an example of a trip. We would start at D6 and end at B4:    

| D | A | C | E | B |
|---|---|---|---|---|
| 6 | 5 | 5 | 3 | 1 |
| 1 | 4 | 3 | 4 | 5 |
| 2 | 3 | 4 | 5 | 6 |
| 4 | 1 | 6 | 1 | 2 |
| 3 | 2 | 2 | 6 | 3 |
| 5 | 6 | 1 | 2 | 4 |

There are 5! permutations of the counties [A, B, C, D, E]. Once we start at a county, D in this example, there are 6! permutations of the towns. Since there are 5 counties that means there are $(6!)^5$ ways to order all of the towns across all counties. We can multiply the permutations of towns since they are independent events. To sum up:  

### $\text{(Ways to order counties) * (Ways to order towns in a county})^{\text{(Number of counties)}}$

## $5!*(6!)^5 = 23,219,011,584,000,000$

# 1.7

Imagine a game of tic-tac-toe where the players randomly select a blank space each turn to make their move. If  $X$  goes first, what is the probability that  $X$  wins on their third move?

In [24]:
# Three moves for X means two moves for O, five total moves
# Setting up an array to index the playing grid and the markers (X = 1, O = -1)
# Also an array to hold the number of times we won
grid_inx = np.arange(9)

markers = np.array([1,1,1,-1,-1], 'int')

result = np.zeros(50000)

# Simulating
for i in range(50000):
    # Resetting the grid
    grid = np.zeros(9, 'int')

    # Shuffling the grid and the markers
    np.random.shuffle(markers)
    np.random.shuffle(grid_inx)

    grid[grid_inx[0:5]] = markers

    grid_sq = grid.reshape(3,3)
    
    # X is represented by 1
    # A sum of 3 across any diagonal or row/col is a win
    if 3 in grid_sq.sum(axis=1) or 3 in grid_sq.sum(axis=0):
        result[i] = 1
        continue
    
    if np.trace(grid_sq) == 3 or np.trace(np.fliplr(grid_sq)) == 3:
        result[i] = 1

np.mean(result) # About 9.5%

0.096579999999999999

### 1.7 Analytical Solution:

Let's start off by thinking about the denominator - the total number of possible games. Since $X$ goes three times, that implies $O$ goes twice. We can think of each move as sequentially sampling the list [X,O,X,O,X]. For the first $X$ there are 9 places it can be drawn. Next, the first $O$ can be placed on any of the 8 remaining squares. The second $X$ has 7 possible locations. Each move lowers the number of possible places the subsequent move can be drawn. Iterating through this logic gives us $9*8*7*6*5 = \frac{9!}{4!} = 15,120$ possible games. 

Now let's think about all of the ways we could win. Three $X$s could show up on a diagonal (2), across a row (3), or across a column (3). That means there are 8 possible ways to win.

For each winning sequence, say three across the first row, there are 3! ways to order the $X$s. Order matters here because each sequence of five turns corresponds to a unique game. Since we're focusing on the $X$s right now, we care about the number of possible ways we could arrive at each winning location (diagonals, rows, or columns). This gives us $8*3! = 48$ ways to place the winning $X$s across all the winning locations.

Now let's count the number of ways the losing $O$s could show up. Once three spots have been covered by $X$, we're left with $\binom{6}{2}$ ways to grab two empty spots for the $O$s. However, the binomial coefficient removes the extra counts of various permutations for a unique set of two spots. In other words, placing $O$ in spots [5,6] and [6,5] counts as the same combination under the binomial coefficient. These actually represent distinct orders of moves that form two different games. We therefore must multiply by 2 so that [5,6] and [6,5] are counted as distinct events. This is equivalent to computing permutations of a subset $\frac{n!}{(n-k)!}$. In this case we've got $\binom{6}{2}*2 = \frac{6!}{(6-2)!} = 30$ permutations of the subset.

To sum up:  
### $\frac{\text{Winning positions * Orderings of each winning position * Possible moves by the other player}}{\text{Total number of distinct games}}$

## $\frac{8*3!*(\binom{6}{2}*2)}{\frac{9!}{4!}} \approx 9.5\%$

# 1.8

Imagine a modified game of tic-tac-toe called ‘tic-tac-max’. The only difference in tic-tac-max is that players put down $X$’s and  $O$’s until the board is completely filled, not just until someone gets 3 in a row. $X$ still goes first.

a. Define a permutation as one game sequence; i.e., player 1 puts an $X$ in the top left spot, then player 2 puts an $O$ in the top right spot, etc. Permutations are considered distinct if they are completed in different orders. How many possible permutations are there to play this game?

b. Define a ‘final board’ as the configuration of the board after each player has put all of their ‘pieces’. How many possible ‘final boards’ are there?

### 1.8.a Analytical Solution:

This is related to how the number of possible games is calculated in problem 1.7. Imagine all of the possible moves in a list [X,O,X,..]. There are 9 places to draw the first move. After the first $X$ has been placed, there are 9-1 spots to place the first $O$. Sequentially looping through the moves and calculating the $(9-move)$ places left to draw an $X$ or $O$ is computationally equivalent to calculating 9!. There are 362,880 possible games of 'tic-tac-max'. This actually overcounts the [true number](http://www.se16.info/hgb/tictactoe.htm) of tic-tac-toe games.

### 1.8.b Analytical Solution:

There are three ways to think about this problem:

1) There are many paths to each final configuration (9!) but a comparatively small set of distinct final configurations. Since $X$ goes first there are 5 $X$s and 4 $O$s. For each final configuration we are overcounting by the number of ways to order the $X$s. We're also overcounting by the number of ways to order the $O$s. That means we have to adjust out the permutations for each player out of the total number of possible games to arrive at the number of final board configurations: 

### $\frac{9!}{5!*4!} = 126$

To illustrate this point further. The following grid counts as one distinct final configuration. However, there are $5!*4!$ ways to get to this game state, which counts towards the total number of permutations:

|   |   |   |
|---|---|---|
| x | x | O |
| O | O | X |
| x | O | x |

2+3) The second approach looks at a blank board and fills in either the $X$ positions **or** the $O$ positions, one group at a time. Picking all of the squares devoted to $X$ (5) and marking the rest (4) for $O$ is equivalent to going the other way around. We're essentially sampling the grid spots without replacement where order doesn't matter. This is related to the symmetry of the binomial coefficient. 

### $\binom{9}{5} = \binom{9}{4} = \frac{9!}{5!*4!} = 126$

# 1.9

There are $2n$ people. How many ways are there to pair the people up?

For example, if we had four people named $\text{A, B , C and D}$ one permutation would be the pairs {A,B}{C,D}. A second permutation would be the pairs {A,C}{B,D} (one permutation is defined by pairing everyone).

### 1.9 Analytical Solution:

The solution is better explained in [this video](https://www.youtube.com/watch?v=VVjCTQ8vujI).

# 1.10

Imagine the digits  0,1,2,...,9. How many ways are there to make a three-digit number and seven-digit number from these digits? For example, {538} and {1246790} as the three- and seven-digit numbers is one permutation; a second permutation is {835} and {1246790}.

### 1.10

This is basically the definition of symmetry of the binomial coefficient. Picking the three-digit number first and making everything else go into the seven-digit number is equivalent to going the other way around.

# $\binom{n}{k} = \binom{n}{n-k}$

# $\binom{10}{3} = \binom{10}{7} = 120$

# 1.11

Imagine five unconnected circles in the shape of a pentagon. 

Imagine selecting two points at random and drawing a straight line in between the two points. Do this 5 times, with the constraint that you cannot select the same pair twice. What is the probability that the lines and points form a pentagon (i.e., a five-sided, five-angled, closed shape)?

In [25]:
# Get all pairs of points
combs = list(it.combinations([1,2,3,4,5],2))

# Setup for simulations
num_sims = 500000

pent = np.zeros(num_sims)

for i in range(num_sims):
    
    rnd.shuffle(combs)

    choices = sorted(combs[0:5])

    if choices == [(1,2),(1,5),(2,3),(3,4),(4,5)]:
        pent[i] = 1

pent.mean()

0.0040020000000000003

### 1.11 Analytical Solution

First, let's construct the total number of possible pairs of points. This is simply the binomial coefficient $\binom{5}{2} = 10$.

Next, we need to pick five of these pairs without replacement. This requires the binomial coefficient again $\binom{10}{5} = 252$.

Since only one of these 252 combinations of five pairs results in a pentagon, the probability is simply $1/252 \approx 0.3968\%$.

To sum up:

### $1 \div \binom{\binom{5}{2}}{5} = 1 \div \binom{10}{5} = 1/252 \approx 0.3968\%$

We could also think of consecutively sampling five pairs from the five original points:

### $\binom{5}{2} * (\binom{5}{2}-1) * (\binom{5}{2}-2) * (\binom{5}{2}-3) * (\binom{5}{2}-4) = 10*8*7*6 = 10!/5!$

However this overcounts the ways to connect pairs of points since order doesn't actually matter. The same five pairs picked in a different order are overcounted by a factor of five. For example, these would count as a distinct group of pairs:

[1,2],[2,4],[2,5],[3,4],[3,5] == [2,4],[2,5],[3,4],[3,5],[1,2]

There are 5! ways to order these pairs, which we have to adjust out:

### $1 \div (10*8*7*6)/5! = 1/252 \approx 0.3968\%$


# 1.12

There are n people, each with a left and a right foot. Each has 2 shoes (one for each foot), so there are 2n shoes. Define a ‘permutation’ as an allocation of the 2n shoes to the n people, such that the left shoes are on the left feet and the right shoes are on the right feet. If shoes are distinguishable, how many permutations are there?

### 1.12 Analytical Solution

Just to clarify, left shoes must go on left feet and right shoes must go on right feet. Left shoes *as a group* are indistinguishable in the sense that any left shoe is OK to wear with any right shoe.

There are n right shoes and n left shoes. Allocating a left shoe to a person is completely independent from allocating a right shoe. There are $n!$ permutations of left shoes and $n!$ permutations of right shoes. Since picking a left shoe is independent from picking a right shoe, there are $n!*n! = (n!)^2$ total permutations of both shoes.

# 1.13 

Nick claims that, when $n > k$, we have $\binom{n}{k}*\binom{k}{n} = 1$ because, by the definition of the binomial coefficient, the multiplication in these terms cancel out. Explain why Nick is wrong, using both math and intuition.

### 1.13 Analytical Solution

The binomial coefficient *is not* the same as the division operator. The binomial coefficient is defined as:

# $\binom{n}{k} = \frac{n!}{(n-k)!(k!)}$

Let's try an example with n = 5 and k = 2. For the left hand side of Nick's multiplication we can see that $\binom{n}{k}$ is:

# $\frac{5!}{(5-2)!(2!)} = \frac{120}{6*2} = 10$

If we try this with the numbers reversed, we can quickly run into some issues:

# $\frac{2!}{(2-5)!(5!)} = \frac{2}{-3!*120} = ???$

First, the binomial coefficient is not defined for negative numbers. There is an *analytic continuation* of the binomial coefficient that covers non-positive integers, the gamma function, but this is outside the scope of this chapter. Even if we use the gamma function this reduces to $\frac{-1}{360}$. This result is far from the first answer.

The main takeaway is that the binomial coefficient does not work like a plain vanilla fraction. They are distinct tools in the mathematics toolbox.

# 1.14

Consider 10 tosses of a fair coin. Let A be the event that you get exactly 5 heads, and B be the event that you get 10 heads.

a. Compare P(A) and P(B).  
b. Now consider the sequences HTHTHTHTHT and HHHHHHHHHH. Compare the probabilities that these sequences occur.  
c. Discuss your results in parts a. and b.

In [26]:
num_sims = 1000000

## Probability of exactly 5 heads
# Set up for simulation
five_heads = np.zeros(num_sims)

for i in range(num_sims):
    
    sample = np.random.binomial(n = 1, p = 0.50, size = 10)
    
    if np.sum(sample) == 5:
        five_heads[i] = 1
        
print('Probability of exactly 5 heads: '+str(np.mean(five_heads)))

## Probability of all heads
# Set up for simulation
all_heads = np.zeros(num_sims)

for i in range(num_sims):
    
    sample = np.random.binomial(n = 1, p = 0.50, size = 10)
    
    if np.sum(sample) == 10:
        all_heads[i] = 1
        
print('Probability of all heads: '+str(np.mean(all_heads)))

Probability of exactly 5 heads: 0.245428
Probability of all heads: 0.00097


### 1.14 Analytical Solution

a. This is almost forcing us to derive the probability mass function for the binomial distribution. Let's start off simple. For the 10 coin flips, there are $\binom{10}{5} = 252$ ways to observe getting exactly 5 heads. There are $n^k = 2^{10} = 1024$ possible sequences of heads and tails that we can observe. Therefore, we have:

## $\frac{\text{Ways to get exactly 5 heads}}{\text{All possible H and T patterns}} = \frac{252}{1024} \approx 0.246$

We can apply the same logic to the second example. There is only one way to observe all heads, so: $\frac{1}{1024} \approx 0.00098$.

To summarize the relationship between the two: $P(\text{5 heads}) = \binom{10}{5}P(\text{All heads})$

If we generalize the logic from above, we can see that for a **fair coin** the probability of observing a sequence with a certain number of heads is:

## $\frac{\binom{trials}{successes}}{2^{\text{trials}}} = 0.5^{\text{successes}}*\binom{trials}{successes}$

This places us very close to the actual probability mass function for any coin, not just fair coins.

b. By the logic laid out above, we can see that both HTHTHTHTHT and HHHHHHHHHH are distinct events. There is **only one way** to observe both. This is not equivalent to observing a certain number of heads or tails. That means:

## P(HTHTHTHTHT) = P(HHHHHHHHHH) = $\frac{1}{1024} \approx 0.00098$

c. The key to these problems is to figure out if we're interested in observing a **certain number of heads** or a **particular pattern**. The latter usually has a much lower probability of occurring.

# 1.15

How many possible 5-letter words are there such that two letters within the word don’t repeat? That is, ‘alamo’ is ok, but ‘aalmo’ is not ok, since the letter ‘a’ repeats.

### 1.15 Analytical Solution

We can rephrase the problem like this:

### $\text{[Any letter] * [Any letter except the previous] * [Any letter except the previous]...}$

Since there are 26 letters in the English alphabet this works out to:

### $\text{[26] * [25] * [25] * [25] * [25]} = 26*25^4 = 10,156,250$

Note that this answer will vary depending on the [alphabet](https://en.wikipedia.org/wiki/List_of_writing_systems#Linear_nonfeatural_alphabets) used.

# 1.16

There is a restaurant in France called ‘7367’. After every meal, the waiter rolls four fair, 10-sided dice (numbered 1 through 10). If the dice show a 3, a 6 and two 7’s (in any order) the meal is free.

a. If you eat at this restaurant once, what is the probability of winning a free meal?

b. This dice game has made the restaurant very popular, but the manager of the restaurant wants to be able to adjust the game. He would like to present variations of the game for meals that are more expensive (lower probability of winning) and for meals that are less expensive (higher probability of winning).

Keeping the same structure of the dice game, how can you change the ‘winning number’ (i.e., 7367) to suit the manager’s needs, both for higher and lower probabilities of winning? Support your answer both with calculations and intuition.

In [27]:
# Setting up empty array to hold results
free_meal = np.zeros(1000000)

for i in range(len(free_meal)):
    
    if list(sorted(np.random.randint(1, 11, 4))) == [3,6,7,7]:
        free_meal[i] = 1
        
np.mean(free_meal)

0.0011739999999999999

In [28]:
# Setting up empty array to hold results
# Let's try a situation where the numbers are unique
free_meal = np.zeros(1000000)

for i in range(len(free_meal)):
    
    if list(sorted(np.random.randint(1, 11, 4))) == [1,2,3,4]:
        free_meal[i] = 1
        
np.mean(free_meal)

0.0023800000000000002

In [29]:
# Setting up empty array to hold results
# Let's try a situation where the numbers are all the same
free_meal = np.zeros(1000000)

for i in range(len(free_meal)):
    
    if list(sorted(np.random.randint(1, 11, 4))) == [7,7,7,7]:
        free_meal[i] = 1
        
np.mean(free_meal)

0.00011900000000000001

### 1.16 Analytical Solution

a. We know that the total number of possible four digit decimal numbers is $10^4 = 10,000$, this is our denominator. The sorted set of numbers needed to win is {3,6,7}. Here we have only three unique numbers, since 7 repeats twice. We can imagine constructing a four digit combination from scratch:

## [A, B, C, D]

We could start by placing the 3 into a spot - there are four currently empty spots. Next, we could place the 6 into any of the remaining three spots. Finally, we must place the **pair** of 7s into the last possible two spots. This is essentially counting permutations where we're counting a pair of a number as one item. This looks like:

#### $\text{[Places to put first unique number (3 or 6)] * [Places to put the second unique number (3 or 6) * [Places to put the pair of 7s]]} = 4*3*1 = 12$

It doesn't matter if we start with the 7s or the {3,6}. If we start with the 7s, we can see there are $\binom{4}{2} = 6$ ways to place the pairs. Then we have two empty spots, which have 2x1 ways to be filled by the {3,6}.

We could also notice that this problem is very similar to the Ivy League School problem's second part (problem 1.2). There are 4! ways to arrange the 4 items. Since two of the items (7s) are not unique, we must adjust out for them. This leaves us with $\frac{4!}{2!} = 12$ unique combinations.

And by the definition of probability:

## $\text{Probability of winning} = \frac{\text{Ways to construct [3,6,7,7]}}{\text{Possible die outcomes}} = \frac{12}{10,000} = 0.0012$

b. There are four ways of adjusting the probabilities:
    1. Increase the 'uniqueness' of the winning numbers, such as [1,2,3,4] --> Easier to win
    2. Decrease the 'uniqueness' of the winning numbers, such as [7,7,7,7] or [1,1,2,2]--> Harder to win
    3. Roll more dice, such as [{1:10},{1:10},{1:10},{1:10},{1:10}] --> Harder to win
    4. Have dice with more sides [{1:16},{1:16},{1:16},{1:16}] --> Harder to win
    
Intuitively, a more unique set of winning numbers has a higher probability of winning. This is because when calculating the permutations of a group that has some repetition, we always have to adjust the number of permutations down. The number of (unordered) ways we can see [1,2,3,4] in the wild is much larger than [7,7,7,7]. The 7s **must** be found as an entire group while the 1:4 could be found individually in any order.

# 1.17

Matt, Dan, Alec, Edward and Patrick are settling in for board game night. They pick their spots randomly at a round table with 5 evenly spaced seats. What is the probability that Dan is sitting next to Alec (i.e., he is sitting either directly to the left or right of Alec)?

In [30]:
# Setting up simulation size
num_sims = 10000

sim_seats = np.zeros(num_sims)

for i in range(num_sims):
    # Randomly creating a seating arrangement
    seats = np.arange(1,6)

    np.random.shuffle(seats)

    # Wrapping array around
    seats = np.append(seats, seats[0])

    # We want 1 and 2 to be next to each other
    # We test for that to see if a rolling 2 element sum is 1 + 2 = 3
    # Nothing else can sum to this
    summed_neighbors = np.convolve(seats, np.ones(2), mode = 'valid')

    if 3 in summed_neighbors:
        sim_seats[i] = 1
        
np.mean(sim_seats)

0.51080000000000003

### 1.17 Analytical Solution

We can visualize the 5 seats as a pentagon with vertices labeled [A,B,C,D,E]. If Alec is sitting at A, there are two places where Dan can sit next to him [E,B]. There are also two places where Dan doesn't sit next to Alec [D,C]. There's nothing particularly special about starting off with A, this 2 next-to vs. 2 not-next-to dynamic is always present. That means that for any seating permutation, there is a 50% chance that Dan is next to Alec.

We can also view this as:

### $\frac{\text{Dan next to Alec}}{\text{Total seating locations for Dan & Alec}} = \frac{\text{5 places for Alec to sit x 2 places for Dan to sit next to him}}{\text{5 places for Alec to sit x 4 places for Dan to sit}} = \frac{10}{20} = 0.5$

# 1.18

Jon Snow and Robb Stark are both in need of knights. There are currently 100 available knights at Winterfell (Jon and Robb’s home); Jon is going to take 10 knights and go North, and Robb is going to take 30 knights and go South.

To avoid potentially displaying favoritism and upsetting the knights of the realm, Jon and Robb will both randomly select their knights. They will flip a coin to determine who goes first; the winner of the coin flip will randomly select his required number of knights (10 or 30) and the loser of the coin flip will then randomly select his required number of knights (10 or 30) from the remaining number of knights (90 or 70).

a. Jon claims that there are ${\binom{100}{10}*\binom{90}{30}}$ possible combinations (one single combination marks the knights that Jon took, the knights that Robb took and the knights that stayed back). Robb, though, claims that Jon is off by a factor of 2, because he forgot to account for who picks their knights first (Jon or Robb). Who is correct?

b. Robb and Jon flip the coin, and Robb wins (the coin lands on the “direwolf” sigil, not the “crow” sigil). However, Robb is not pleased: “Great”, he says, “now I go first, which means I have a higher chance of taking Theon” (Theon is one of the 100 knights and is well known for his ineptitude). Robb and Jon will still both randomly select their knights, as the rules stipulate; does Robb have a higher chance of selecting Theon if he goes first?

### 1.18.a Analytical Solution

This is a twist on the symmetry of the binomial coefficient. It will be easier to see this by working through the algebra:

 N = total number of knights = 100  
 A = size of first group = 10  
 B = size of second group = 30  
 
 Option (1) = Pick A then B
 
 Option (2) = Pick B then A
 
 ## $\binom{100}{10}*\binom{90}{30} \text{  vs.  } \binom{100}{30}*\binom{70}{10}$
 
 Option (1):
 
 ## $\frac{N!}{(N-A)!*A!}*\frac{(N-A)!}{(N-A-B)!*B!} = \frac{N!}{(N-A-B)!*A!*B!}$
 
 Option (2):
 
 ## $\frac{N!}{(N-B)!*B!}*\frac{(N-B)!}{(N-B-A)!*A!} = \frac{N!}{(N-B-A)!*B!*A!}$
 
 We can see that going either way is equivalent, Jon is correct. The set of combinations is exactly the same between the two paths.

### 1.18.b Analytical Solution

Since the set of combinations is exactly the same, the chances of Robb picking Theon should be equivalent under both scenarios. 

Let's first imagine picking 40 random knights from the larger group of 100. This is what Robb and Jon will take 10 and 30 out of. The probability of the group of 40 containing Theon is 40%. We can arbitrarily assign each knight from the 40 to the 10 and 30 groups. 

The probability of Robb picking Theon is therefore 40% * 30/(10+30) = 30%. This is true regardless of who picks first.

In [32]:
# Probability of finding Theon in a group of 40
# Sanity check
sims = np.zeros(10000)

for i in range(len(sims)):
    if 1 in np.random.choice(np.arange(1,101), 40, replace = False):
        sims[i] = 1
    
np.mean(sims)

0.3972

In [34]:
# Probability of Theon being picked under both scenarios
num_sims = 10000

# Robb first, he picks first 30 out of random 40 group
robb_theon = np.zeros(num_sims)

for i in range(len(robb_theon)):
    
    knights = np.random.choice(np.arange(1,101), 40, replace = False)

    robb_first = knights[0:30]
    
    if 1 in robb_first:
        robb_theon[i] = 1
        
print(np.mean(robb_theon))

# Jon first, Robb second - he picks last 30 out of random 40
robb_theon = np.zeros(num_sims)

for i in range(len(robb_theon)):
    
    knights = np.random.choice(np.arange(1,101), 40, replace = False)

    robb_second = knights[10:41]
    
    if 1 in robb_second:
        robb_theon[i] = 1
        
print(np.mean(robb_theon))

0.3025
0.3041


# 1.19

Imagine a game called ‘Cards Roulette’. The four Aces are removed from the deck and are well shuffled. You take turns with a friend drawing Aces without replacement from the four cards. The first player to draw a red Ace (Hearts or Diamonds) loses.

a. If you would like to maximize your probability of winning this game, should you go first or second?

b. Compare this to the Russian Roulette problem from this chapter (click [here](https://www.youtube.com/watch?v=qLoFGDfJ9wA) for a video recap of this problem). Think about how these two problems compare in structure and make an intuitive argument comparing the two solutions.

In [35]:
# Tagging the cards as [1,2,3,4]
# Red cards are even [2,4], so if you pick an even card you lose
num_sims = 10000

# Initializing all games as winning
go_first  = np.ones(num_sims)
go_second = np.ones(num_sims)

# Calculating probability of losing at each turn
for i in range(num_sims):
    # Shuffling cards
    cards = np.random.choice(np.arange(1,5), 4, replace = False)
    
    if cards[0] % 2 == 0:
        go_first[i] = 0
        continue
    
    if cards[1] % 2 == 0:
        go_second[i] = 0
        continue
        
    if cards[2] % 2 == 0:
        go_first[i] = 0
        continue
    
    # Will never get to this turn
    if cards[3] % 2 == 0:
        print('Fourth turn?!')
        go_second[i] = 0
        continue

# Probability of winning
print(np.mean(go_first))
print(np.mean(go_second))

0.3335
0.6665


### 1.19 Analytical Solution

a. There are $4! = 24$ permutations of the cards [Black-1, Black-2, Red-1, Red-2].

Let's calculate the probability of the first player losing either on the first or third turn.

On the first turn all we need is a red card in the first slot to make the first player lose. [Red-x, ?, ?, ?]. There are $2*3! = 12$ ways to construct this sequence. We have two red aces and there are 3! permutations for the remaining cards.

If the first and second player both manage to survive the first two rounds, the second player will definitely lose on the third round. The previous two rounds must have been black aces, which only leaves red cards. This looks like [Black-x, Black-x, Red-x, Red-x]. There are $2!*2! = 4$ ways to get here. The two black and red aces are interchangeable.

Therefore, the probability of the first player *losing* is:

## $\frac{2*3! + 2!*2!}{4!} = \frac{16}{24} \approx 0.6667$

Since one of the players must win, the probability of the second player *winning* is $\approx 0.6667$. The probability of the first player *winning* is $\approx 0.333$.

b. This is strictly different from the Russian Roulette problem. A general approach to probability involves thinking about a string of events as being part of a garden of forking paths. Counting the number of favorable paths and dividing by the total number of paths gives a basic estimate of probability.

The Russian Roulette problem can be rephrased as 'ownership' over six chambers by two players. Each chamber has an equal probability of containing the bullet. Intuitively, the order of turns doesn't matter. All that matters is who owns the three chambers in which one has a bullet. The garden of forking paths here is very simple. We just have two nodes:
    1. Bullet in chambers {1,3,5}?
    2. Bullet in chambers {2,4,6}?

The Cards Roulette problem cannot be rephrased in terms of 'ownership' of slots. Observing a black card on the first turn creates a new layer in the garden of forking paths. This impacts the number of ways the second player can lose or win. New information changes how we count the probabilities of subsequent events. Observing a black card first directly lowers the probability of observing a second black card. The garden looks like:
    * Red card in slot 1?
        a. [Yes] - First loses {12 ways}
        b. [No] - Red card in slot 2? {12 total ways}
            I. [Yes] - First wins {8 ways}
            II. [No] - First loses {4 ways}

The garden stops being symmetric on the third level, this breaks the symmetry we see in the Russian Roulette problem.

It would be interesting to see a twist on this game that allows players to change the slots that belong to them after seeing the first card. This is similar to the [Monty Hall Problem](http://mathworld.wolfram.com/MontyHallProblem.html). 

# 1.20

Using a story proof, show:

## ${n \choose x} 2^x = \frac{\big(2n\big)\big(2n - 2\big)...\big(2n - 2(x - 1)\big)}{x!}$

Where $n > x$.

Hint: During police interrogations, it is common to adapt a ‘good cop/bad cop’ strategy. That is, two cops enter the interrogation room, and one cop is the ‘good cop’ (they are nice to the perpetrator to try to get them to open up) while the other cop is the ‘bad cop’ (they are rude and possibly even threatening to scare the perpetrator).

Imagine that you are the police commissioner and you are presented with pairs of cops; the cops come in teams of two. It is your job to select teams, and then assign which cops will be the ‘good cops’ and which cops will be the ‘bad cops’ (each team must have one good cop and one bad cop).

### 1.20 Analytical Solution

Let's first work through the algebra to prove these are actually equivalent. Let's expand the first few terms of some of the factorials in the numerator and denominator. Let's also spread out that $2^x$ into the underlying sequence.

## $\binom{n}{x}*2^x = \frac{\text{2*n * 2*(n-1) * 2*(n-2)...}}{\text{(n-x) * (n-x-1) * (n-x-2)... * x!}}$

Since n is greater than x, we know that all of components of the (n-x)! expansion live in the numerator. Since we're dividing, these cancel out and we're left with:

## $\binom{n}{x}*2^x = \frac{\text{2*n * 2*(n-1) * 2*(n-2) ... 2*(n-k+1})}{x!} = \frac{\big(2n\big)\big(2n - 2\big)...\big(2n - 2(x - 1)\big)}{x!}$

The explanation in the [solution text](https://bookdown.org/probability/solutions2/Probability%21__Solutions_.html) is well written. I am reproducing it below for easier access:

As prompted in the Hint, both sides count the number of ways that the police commissioner can choose teams of cops and then assign a ‘good cop’ and a ‘bad cop’ within each team. Imagine that there are $n$ pairs of cops and the commissioner needs to choose $x$ pairs of cops.

First, consider the LHS. The commissioner first selects the pairs that he will assign. There are nn total pairs and he will select $x$ pairs, so there are $\binom{n}{x}$ possible choices. After he has selected the pairs, he needs to assign a good cop and a bad cop in each pair. There are 2 options for each of the $x$ pairs (either cop can be the good cop), so by the multiplication rule there are $2x$ possible combinations once the pairs have been decided. Putting it all together via the multiplication rule, we have $\binom{n}{x}2^x$, as desired.

Next, consider the RHS. Here, the police commissioner picks a random cop to be a good cop, and then automatically assigns that cop’s partner as the bad cop of that pair; these two cops make the first pair. With the first pair gone, he then randomly selects another good cop, and proceeds in the same way. The first choice has $2n$ possibilities (he can choose any of the cops), the second choice has $2n−2$ possibilities (he can choose any of the cops except for the first pair), etc., until the commissioner has chosen xx pairs. By the multiplication rule, we have $\big(2n\big)\big(2n - 2\big)...\big(2n - 2(x - 1)\big)$, or the numerator on the RHS. This overcounts by a factor of x!x! though, as it does not matter the order that the commissioner picks cops in; he can arrive at the same combination in different orders. Dividing this factor of $x!$ out gives us the RHS.

Ultimately, the LHS considers when the commissioner picks the pairs first and then the good cops, and the RHS considers simply when the commissioner picks good cops and automatically assigns pairs in that way.

# 1.21

Define ‘skip-counting’ as an alternative way to count integers. Imagine counting from 1 to 10: a legal ‘skip-count’ is an ascending set of integers that starts with 1 and ends with 10 (i.e., 1-2-5-6-10 and 1-3-10 are both legal ‘skip-counts’).

How many skip-counts are there if we skip-count from 1 to some integer $n>1$?

### 1.21 Analytical Solution

Let's think about when n is 4. Here are all of the possible skip counts:

#### 1 2 3 4 || 1 2 4 || 1 3 4 || 14

As you increase n above 4 you notice a pattern emerge. We know that {1,n} will **always** be present, this is by design. The only thing that changes is the number of numbers between the floor and ceiling. We know that the largest sequence is the most obvious one, just counting from 1 to n. Let's pretend that these integers will always be sorted but we can pick to turn "off" some of the integers in the largest sequence:

[  1 - ON  ||  2 - OFF  ||  3 - ON  ||  4 - OFF  ||  5 ON  ||] = [1,3,5]

We really only have the ability to turn "off" or "on" everything except {1,n}. This gives us (n-2) binary levers we can pull. We're essentially sampling [ON,OFF] with replacement where order matters. This leaves results in the following number of skip counts:

## $2^\text{total numbers - tails we can't touch} = 2^{(n-2)}$

# 1.22

The NFL (National Football League) consists of 32 teams that play 17 games amongst each other in a season (each team is given one ‘bye’ week that they have off, but for simplicity in this problem we will assume all teams play all 17 weeks). By extension, there are 16 games each week (each of the 32 teams plays one other team).

Imagine entering an NFL betting pool with the following rules: you can purchase a single entry for $1, which allows you to pick a game a week until you are wrong. That is, every week, you select a team out of the 32 that you think will win, and if you are correct (the team wins or ties) you advance to the second week. You win if you ‘survive’ for 17 weeks (pick 17 games, one in each week, correctly).

You are allowed to purchase multiple entries; simply imagine that different entries are different chances to play the game. Each entry is independent (you can pick different games, the same games, etc. with different entries) and when an entry fails (you pick a wrong game for that entry) the entry is eliminated, but other entries may continue. If any one of your entries makes it 17 weeks, you win.

What is the least amount of money you have to pay to buy enough entries that allow you to implement a strategy that guarantees victory (at least one entry survives)?

### 1.22 Analytical Solution

Let's assume that team 1 plays team 2 every week. We know for a fact that one team must win each week. After 3 weeks, the winners might be:

#### [1, 2, 1]

Picking a correct winning path gets progressively more difficult as the number of weeks increases. For any week there are 2 possible outcomes, which compounds with the possible paths of the previous week. Here's what all of the paths over three weeks look like:

1. [1,1,1]
2. [1,1,2]
3. [1,2,2]
4. [2,2,2]
5. [2,2,1]
6. [2,1,1]
7. [1,2,1]
8. [2,1,1]

We're essentially sampling winners from [1,2] with replacement where order matters. This is equivalent to $2^n$ where n is the length of the sequence of binary events. For this specific problem this turns out to be $2^{17} = \$131,072$.

$2^n$ also gives us the size of the [**power set**](https://en.wikipedia.org/wiki/Power_set). It gives the number of ways to slice up a set of something. This includes the entire set, as well as the empty (null) set. This comes up in probability, set theory, and computer science.

An interesting twist on this problem deals with betting increasing quantities of money on a string of binary successes. 